# Visitor
> Represent an operation to be performed on the elements of an object structure. Visitor lets you define a new operation without changing the classes of the elements on which it operates.

## Problem
Lets say I have two types of employess - `Contractor`s and `FTE`s. 

In [19]:
# These classes are in a 3rd party library.
from abc import ABC

class Employee(ABC):
    pass

class Contractor(Employee):
    def __init__(self, name, wage, hours_worked):
        self.name = name
        self.wage = wage
        self.hours_worked = hours_worked


class FullTime(Employee):
    def __init__(self, name, salary):
        self.name = name
        self.salary = salary

In [8]:
# I use these classes all over my app.
employees = [
    Contractor("Harry Potter", 10.50, 100), 
    Contractor("Draco Malfoy", 17.99, 10), 
    FullTime("Hermione Granger", 100_000)
]

Now one day I needed to calculate the quarterly pay for all my employees. My typical approach would be to just write a simple module that does just that.

In [6]:
def qtr_pay(employees):
    for employee in employees:
        if isinstance(employee, Contractor):
            pay = employee.wage * employee.hours_worked
        elif isinstance(employee, FullTime):
            pay = employee.salary / 4
        print(f"{employee.name} earned ${pay:.2f} this quarter.")

qtr_pay(employees)

Harry Potter earned $1.0e+03 this quarter.
Draco Malfoy earned $1.8e+02 this quarter.
Hermione Granger earned $2.5e+04 this quarter.


However, I might think, "hmm...this new functionality to calculate quarterly pay is pretty useful. There might be other places this can be used." In this case I might just decide to add a new method to both my classes and their interface be done with that. 

In [45]:
class Employee(ABC):
    @abstractmethod
    def calc_qtr_pay(self):
        pass
    
class Contractor(Employee):
    def __init__(self, name, wage, hours_worked):
        self.name = name
        self.wage = wage
        self.hours_worked = hours_worked
    
    def calc_qtr_pay(self):
        return self.wage * self.hours_worked


class FullTime(Employee):
    def __init__(self, name, salary):
        self.name = name
        self.salary = salary
        
    def calc_qtr_pay(self):
        return self.salary / 4
    

In [46]:
employees = [
    Contractor("Harry Potter", 10.50, 100), 
    Contractor("Draco Malfoy", 17.99, 10), 
    FullTime("Hermione Granger", 100_000)
]

for employee in employees:
    pay = employee.calc_qtr_pay()
    print(f"{employee.name} earned ${pay:.2f} this quarter.")

Harry Potter earned $1050.00 this quarter.
Draco Malfoy earned $179.90 this quarter.
Hermione Granger earned $25000.00 this quarter.


At least I got rid of the nasty `isinstace` call from my report generator. But this is still too brittle because I have to change multiple modules, one for each subclass. How can I avoid that?

## Solution

### First Attempt
Let me think of this new functionality not as a method, but as a class in itself.

In [47]:
class QtrPay:
    def _calc_contractor(self, employee):
        return employee.wage * employee.hours_worked
    
    def _calc_fte(self, employee):
        return employee.salary / 4
    
    def calc(self, employee):
        if isinstance(employee, Contractor):
            return self._calc_contractor(employee)
        elif isinstance(employee, FullTime):
            return self._calc_fte(employee)

And lets re-implement the employee heirarchy the way it was so Jupyter is not confused.

In [48]:
class Employee(ABC):
    pass

class Contractor(Employee):
    def __init__(self, name, wage, hours_worked):
        self.name = name
        self.wage = wage
        self.hours_worked = hours_worked


class FullTime(Employee):
    def __init__(self, name, salary):
        self.name = name
        self.salary = salary

In [26]:
employees = [
    Contractor("Harry Potter", 10.50, 100), 
    Contractor("Draco Malfoy", 17.99, 10), 
    FullTime("Hermione Granger", 100_000)
]

qtr_pay = QtrPay()
for employee in employees:
    pay = qtr_pay.calc(employee)
    print(f"{employee.name} earned ${pay:.2f} this quarter.")

Harry Potter earned $1050.00 this quarter.
Draco Malfoy earned $179.90 this quarter.
Hermione Granger earned $25000.00 this quarter.


### Sidebar: Double Distpatch
In most compiled languages there would be no need for the `isinstance` operator in the `QtrPay` class. There would be overloaded methods like so -
```C#
class QtrPay {
    double Calc(Contractor emp) {
        return emp.Wage * emp.HoursWorked;
    }
    
    double Calc(FullTime emp) {
        return emp.Salar / 4;
    }
    
    double Calc(Employee emp) {
        return -1;
    }
}

static void Main(string[] args) {
    Employee hp = Contractor("Harry Potter", ...);
    Employee dm = Contractor("Draco Malfoy", ...);
    Employe hg = FullTime("Hermione Granger", ...);
    List<Employee> employees = new List<Employee> { hp, dm, hg };
    
    QtrPay qp = new QtrPay();
    foreach (Employee employee in employees) {
        double pay = qp.Calc(employee);
    }
}
```
However, this will not work as expected, `qp.Calc(employee)` will end up calling `QtrPay::Calc(Employee emp)` method every time instead of the expected polymorphic behavior. This is because the compiler opts for static dispatching instead of dynamic dispatching via vtables. To get around this we can use the concept of double dispatching, where instead of calling a method on `qp` object and passing it an instance of `Employee` we call a method on the `employee` object passing it an instance of `QtrPay`.

```C#
interface Employee {
    public double Accept(QtrPay qp);
}

class Contractor : Employee {
    // rest of the implementation remains the same
    public double Accept(QtrPay qp) {
        return qp.Calc(this);
    }
}

class FullTime : Employee {
    // rest of the impl remains
    public double Accept(QtrPay qp) {
        return qp.Calc(this);
    }
}

static void Main(string[] args) {
    Employee hp = Contractor("Harry Potter", ...);
    Employee dm = Contractor("Draco Malfoy", ...);
    Employe hg = FullTime("Hermione Granger", ...);
    List<Employee> employees = new List<Employee> { hp, dm, hg };
    
    QtrPay qp = new QtrPay();
    foreach (Employee employee in employees) {
        double pay = employee.Accept(qp);
    }
}
```

Now, the compiler can still use dynamic dispatching and call the `Accept` method on the concrete subclass which in turn calls the overloaded `Calc` method with the full subclass so again the compiler can do dynamic dispatching, eventually resulting in the right overloaded method being called.

### Second Attempt
Ok so far so good. But I am back to using the nasty `isinstance` call. How to avoid that? If I can change the Employee class heirarchy and add just one more method there called `accept` I can get rid of `isisntace` in QtrPay.

In [35]:
class Employee(ABC):
    @abstractmethod
    def accept(self, visitor: QtrPay):
        pass
    
class Contractor(Employee):
    def __init__(self, name, wage, hours_worked):
        self.name = name
        self.wage = wage
        self.hours_worked = hours_worked
        
    def accept(self, visitor: QtrPay):
        return visitor.calc_contractor(self)
        
class FullTime(Employee):
    def __init__(self, name, salary):
        self.name = name
        self.salary = salary
        
    def accept(self, visitor: QtrPay):
        return visitor.calc_fulltime(self)
        
class QtrPay:
    def calc_contractor(self, emp):
        return emp.wage * emp.hours_worked
    
    def calc_fulltime(self, emp):
        return emp.salary / 4

In [38]:
employees = [
    Contractor("Harry Potter", 10.50, 100), 
    Contractor("Draco Malfoy", 17.99, 10), 
    FullTime("Hermione Granger", 100_000)
]

qtr_pay = QtrPay()
for employee in employees:
    pay = employee.accept(qtr_pay)
    print(f"{employee.name} earned ${pay:.2f} this quarter.")

Harry Potter earned $1050.00 this quarter.
Draco Malfoy earned $179.90 this quarter.
Hermione Granger earned $25000.00 this quarter.


### Third Attempt
Now what if I want to calculate yearly pay? Well I can simply create a different class for this new functionality called `YearlyPay` and as long as it confirms to the same interface as `QtrPay`, i.e., have `calc_contractor` and `calc_fulltime` methods on it, I should be fine. And for that it is a good idea to explicitly define a common interface for these `PayCalculators`. Typically this is called the `Visitor` interface.

In [40]:
from typing import Any

class PayCalculator(ABC):
    @abstractmethod
    def calc_contractor(self, emp: Employee) -> Any:
        pass
    
    @abstractmethod
    def calc_fulltime(self, emp: Employee) -> Any:
        pass
    
class Employee(ABC):
    @abstractmethod
    def accept(self, visitor: PayCalculator) -> Any:
        pass

The rest of the class definitions remain the same. Here they are reproduced for clarity -

In [41]:
class Contractor(Employee):
    def __init__(self, name, wage, hours_worked):
        self.name = name
        self.wage = wage
        self.hours_worked = hours_worked
        
    def accept(self, visitor: QtrPay):
        return visitor.calc_contractor(self)
        
class FullTime(Employee):
    def __init__(self, name, salary):
        self.name = name
        self.salary = salary
        
    def accept(self, visitor: QtrPay):
        return visitor.calc_fulltime(self)

In [42]:
class QtrPay:
    def calc_contractor(self, emp):
        return emp.wage * emp.hours_worked
    
    def calc_fulltime(self, emp):
        return emp.salary / 4
    
class YearlyPay:
    def calc_contractor(self, emp):
        return emp.wage * emp.hours_worked
    
    def calc_fulltime(self, emp):
        return emp.salary

In [44]:
employees = [
    Contractor("Harry Potter", 10.50, 100), 
    Contractor("Draco Malfoy", 17.99, 10), 
    FullTime("Hermione Granger", 100_000)
]

qtr_pay = QtrPay()
for employee in employees:
    pay = employee.accept(qtr_pay)
    print(f"{employee.name} earned ${pay:.2f} this quarter.")
    
yr_pay = YearlyPay()
for employee in employees:
    pay = employee.accept(yr_pay)
    print(f"{employee.name} earned ${pay:.2f} this year.")

Harry Potter earned $1050.00 this quarter.
Draco Malfoy earned $179.90 this quarter.
Hermione Granger earned $25000.00 this quarter.
Harry Potter earned $1050.00 this year.
Draco Malfoy earned $179.90 this year.
Hermione Granger earned $100000.00 this year.


I personally prefer the solution described in the First Attempt. Yes, it has the bad `isinstance` function, but seems much more easy to reason about, i.e., it is self contained enough that anybody looking at that code can figure out what is happening without having to trace the code path through mulitple modules.